In [13]:
# the access key
client_id = "fd156f449b1a44359bb23cf0c2d67dfc"
client_secret = "3a69b492b52b4624b60b620b39e8d4ea"

In [14]:
# Code to get access token 

import requests
import base64

def get_access_token(client_id, client_secret):
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_header = {
        'Authorization': f'Basic {base64.b64encode((client_id + ":" + client_secret).encode()).decode()}'
    }
    auth_data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret
    }
    auth_response = requests.post(auth_url, data=auth_data, headers=auth_header)
    auth_response_data = auth_response.json()
    access_token = auth_response_data['access_token']
    return access_token


In [15]:
def get_genres(access_token, limit=50, offset=0):
    genres_url = 'https://api.spotify.com/v1/browse/categories'
    headers = {
        'Authorization': 'Bearer ' + access_token
    }
    params = {
        'limit': limit,
        'offset': offset
    }
    response = requests.get(genres_url, headers=headers, params=params)
    genres_data = response.json()
    genres = [(category['id'], category['name']) for category in genres_data['categories']['items']]
    return genres


In [16]:
access_token = get_access_token(client_id, client_secret)  
genres = get_genres(access_token)  

print("List of genres:")
genre_list = []
for genre_id, genre_name in genres:
    genre_list.append(genre_name)

print(genre_list)


List of genres:
['Made For You', 'New Releases', 'Summer', 'Country', 'Hip-Hop', 'Pop', 'Charts', 'GLOW', 'Black Music Month', 'Latin', 'Student', 'Discover', 'Rock', 'Dance/Electronic', 'In the car', 'Mood', 'Indie', 'R&B', 'Christian & Gospel', 'Workout', 'Disney', 'Música Mexicana', 'K-pop', 'Chill', 'Netflix', 'Sleep', 'Party', 'At Home', 'Decades', 'Love', 'Metal', 'Spotify CLASSICS', 'Jazz', 'Trending', 'Frequency', 'Classical', 'Folk & Acoustic', 'Focus', 'Soul', 'Kids & Family', 'Gaming', 'Anime', 'TV & Movies', 'Instrumental', 'Punk', 'Ambient', 'Blues', 'Cooking & Dining', 'Alternative', 'Travel']


In [17]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_id = "fd156f449b1a44359bb23cf0c2d67dfc"
client_secret = "3a69b492b52b4624b60b620b39e8d4ea"

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_artist_genres(artist_id):
    artist = sp.artist(artist_id)
    return artist['genres']

access_token = get_access_token(client_id, client_secret)  
def get_artist_genre(artist_id):
    artist = sp.artists(artist_id)
    return artist['genres']

In [20]:
import pandas as pd
import time

def create_dataframe(query_list, limit=50):  # The function create_dataframe(query_list, limit=50) is defined.
    # It takes a list of queries (in this case, genres) and an optional limit for the number of tracks to be fetched for each genre.
    data = {
        'Track ID': [],
        'Track name': [],
        'Artist name': [],
        'Popularity score': [],
        'Release year': [],
        'Genre': []
    }
    # Inside the function, an initial empty dictionary data is created with keys as column names of the dataframe and values as empty lists.
    for query in query_list:
        # Fetch the first 'limit' tracks for each genre
        search_results = sp.search(query, limit=limit, type='track')
        tracks = search_results['tracks']['items']
        
        # Extract the desired information for each track and store it in the data dictionary
        for track in tracks:  # The function then enters a loop, where for each genre in the query list, it performs a search using Spotify’s API, specif
            data['Track ID'].append(track['id'])
            data['Track name'].append(track['name'])
            data['Artist name'].append(track['artists'][0]['name'])
            data['Popularity score'].append(track['popularity'])
            data['Release year'].append(track['album']['release_date'][:4])
            data['Genre'].append(query)

                
    df = pd.DataFrame(data)
    return df


# List of genres
genres = [
    'Made For You', 'New Releases', 'Summer', 'Country', 'Hip-Hop', 'Pop', 'Charts', 'GLOW', 'Black Music Month', 
    'Latin', 'Student', 'Discover', 'Rock', 'Dance/Electronic', 'In the car', 'Mood', 'Indie', 'R&B', 
    'Christian & Gospel', 'Workout', 'Disney', 'Música Mexicana', 'K-pop', 'Chill', 'Netflix', 'Sleep', 
    'Party', 'At Home', 'Decades', 'Love', 'Metal', 'Spotify CLASSICS', 'Jazz', 'Trending', 'Frequency', 
    'Classical', 'Folk & Acoustic', 'Focus', 'Soul', 'Kids & Family', 'Gaming', 'Anime', 'TV & Movies', 
    'Instrumental', 'Punk', 'Ambient', 'Blues', 'Cooking & Dining', 'Alternative', 'Travel'
]

# Create the DataFrame with 50 tracks from each genre
spotify_df = create_dataframe(genres, limit=50) # Accordingly, the function create_dataframe() is called with the genres and limit as parameters, and the resulting dataframe df contains the track information.



In [22]:
from IPython.display import display
display(spotify_df)

spotify_df.to_csv('spotify.csv', index=False)


,Track ID,Track name,Artist name,Popularity score,Release year,Genre
0,1uRQzm3P0nRGnSfnuzMTpq,Made For You,OneRepublic,34,2009,Made For You
1,07q0QVgO56EorrSGHC48y3,I Was Made For Lovin' You,KISS,79,1979,Made For You
2,0WIv5qV41y6YjjB9V1biuC,Made For Me,Muni Long,77,2023,Made For You
3,7vF3xkCMvZjAe2nTWY0uQZ,Made For You,Jake Owen,65,2019,Made For You
4,7zVCrzzEJU7u24sbJPXA5W,What Ifs (feat. Lauren Alaina),Kane Brown,69,2016,Made For You
...,...,...,...,...,...,...
2495,73IlJ59HkcXNNhLp0oBugd,Traveling,Jey Beatz,26,2024,Travel
2496,5cWIRgigrzjTjDBNXQxhAR,Travelling Riverside Blues - 29/6/69 Top Gear;...,Led Zeppelin,38,2016,Travel
2497,4aDDlJRc1WbIlFqhc4aMgx,Calming White Noise Generator,The White Noise Travelers,62,2022,Travel
2498,3ORNkXea1Uk8NdhwU4OB0B,Traveling Poor Boy,Brent Cobb,25,2016,Travel


In [24]:
spotify_df["Genre"].value_counts()

Genre
Made For You          50
Focus                 50
At Home               50
Decades               50
Love                  50
Metal                 50
Spotify CLASSICS      50
Jazz                  50
Trending              50
Frequency             50
Classical             50
Folk & Acoustic       50
Soul                  50
New Releases          50
Kids & Family         50
Gaming                50
Anime                 50
TV & Movies           50
Instrumental          50
Punk                  50
Ambient               50
Blues                 50
Cooking & Dining      50
Alternative           50
Party                 50
Sleep                 50
Netflix               50
Chill                 50
Summer                50
Country               50
Hip-Hop               50
Pop                   50
Charts                50
GLOW                  50
Black Music Month     50
Latin                 50
Student               50
Discover              50
Rock                  50
Dance/Electronic   

In [25]:
def get_audio_features(track_ids):
    features_list = []  # an empty list created to run a loop through all track IDs to retrieve audio features

    for i in range(0, len(track_ids), 50):  # This sets up a loop that iterates over the track IDs in groups of 50.
        batch = track_ids[i:i+50]  # This creates a "batch" of 50 track IDs from the list
        features_results = sp.audio_features(batch)  # calling a function to retrieve audio features of each track
        features_list += features_results  # The result is a list of dictionaries, where each dictionary contains the audio features of a song.

    return features_list


In [27]:
track_ids = spotify_df['Track ID'].tolist()
audio_features = get_audio_features(track_ids)


In [35]:
# Remove None entries from the audio_features list
audio_features = [feature for feature in audio_features if feature is not None]


audio_features_df = pd.DataFrame(audio_features)
merged_df = spotify_df.merge(audio_features_df, left_on='Track ID', right_on='id', how='inner')


In [36]:
merged_df

,Track ID,Track name,Artist name,Popularity score,Release year,Genre,danceability,energy,key,loudness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,1uRQzm3P0nRGnSfnuzMTpq,Made For You,OneRepublic,34,2009,Made For You,0.6400,0.555,7,-10.204,...,0.0981,0.3950,122.038,audio_features,1uRQzm3P0nRGnSfnuzMTpq,spotify:track:1uRQzm3P0nRGnSfnuzMTpq,https://api.spotify.com/v1/tracks/1uRQzm3P0nRG...,https://api.spotify.com/v1/audio-analysis/1uRQ...,257493,4
1,07q0QVgO56EorrSGHC48y3,I Was Made For Lovin' You,KISS,79,1979,Made For You,0.7680,0.852,4,-6.215,...,0.0723,0.8670,128.373,audio_features,07q0QVgO56EorrSGHC48y3,spotify:track:07q0QVgO56EorrSGHC48y3,https://api.spotify.com/v1/tracks/07q0QVgO56Eo...,https://api.spotify.com/v1/audio-analysis/07q0...,271240,4
2,0WIv5qV41y6YjjB9V1biuC,Made For Me,Muni Long,77,2023,Made For You,0.7960,0.468,7,-6.375,...,0.1180,0.0677,119.980,audio_features,0WIv5qV41y6YjjB9V1biuC,spotify:track:0WIv5qV41y6YjjB9V1biuC,https://api.spotify.com/v1/tracks/0WIv5qV41y6Y...,https://api.spotify.com/v1/audio-analysis/0WIv...,188000,4
3,7vF3xkCMvZjAe2nTWY0uQZ,Made For You,Jake Owen,65,2019,Made For You,0.5810,0.441,1,-6.829,...,0.1110,0.3370,82.125,audio_features,7vF3xkCMvZjAe2nTWY0uQZ,spotify:track:7vF3xkCMvZjAe2nTWY0uQZ,https://api.spotify.com/v1/tracks/7vF3xkCMvZjA...,https://api.spotify.com/v1/audio-analysis/7vF3...,238400,4
4,7zVCrzzEJU7u24sbJPXA5W,What Ifs (feat. Lauren Alaina),Kane Brown,69,2016,Made For You,0.6120,0.799,10,-4.603,...,0.1720,0.6870,125.976,audio_features,7zVCrzzEJU7u24sbJPXA5W,spotify:track:7zVCrzzEJU7u24sbJPXA5W,https://api.spotify.com/v1/tracks/7zVCrzzEJU7u...,https://api.spotify.com/v1/audio-analysis/7zVC...,188253,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2544,73IlJ59HkcXNNhLp0oBugd,Traveling,Jey Beatz,26,2024,Travel,0.7680,0.705,11,-6.026,...,0.3290,0.3780,127.918,audio_features,73IlJ59HkcXNNhLp0oBugd,spotify:track:73IlJ59HkcXNNhLp0oBugd,https://api.spotify.com/v1/tracks/73IlJ59HkcXN...,https://api.spotify.com/v1/audio-analysis/73Il...,136934,4
2545,5cWIRgigrzjTjDBNXQxhAR,Travelling Riverside Blues - 29/6/69 Top Gear;...,Led Zeppelin,38,2016,Travel,0.4760,0.780,7,-9.009,...,0.2600,0.6340,91.609,audio_features,5cWIRgigrzjTjDBNXQxhAR,spotify:track:5cWIRgigrzjTjDBNXQxhAR,https://api.spotify.com/v1/tracks/5cWIRgigrzjT...,https://api.spotify.com/v1/audio-analysis/5cWI...,310907,4
2546,4aDDlJRc1WbIlFqhc4aMgx,Calming White Noise Generator,The White Noise Travelers,62,2022,Travel,0.0616,0.350,1,-33.531,...,0.1120,0.0783,71.100,audio_features,4aDDlJRc1WbIlFqhc4aMgx,spotify:track:4aDDlJRc1WbIlFqhc4aMgx,https://api.spotify.com/v1/tracks/4aDDlJRc1WbI...,https://api.spotify.com/v1/audio-analysis/4aDD...,273500,3
2547,3ORNkXea1Uk8NdhwU4OB0B,Traveling Poor Boy,Brent Cobb,25,2016,Travel,0.6930,0.530,4,-9.928,...,0.1050,0.5190,102.795,audio_features,3ORNkXea1Uk8NdhwU4OB0B,spotify:track:3ORNkXea1Uk8NdhwU4OB0B,https://api.spotify.com/v1/tracks/3ORNkXea1Uk8...,https://api.spotify.com/v1/audio-analysis/3ORN...,214160,4


In [37]:
unique_df = merged_df.drop_duplicates(subset='Track ID', keep='first')

unique_df

,Track ID,Track name,Artist name,Popularity score,Release year,Genre,danceability,energy,key,loudness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,1uRQzm3P0nRGnSfnuzMTpq,Made For You,OneRepublic,34,2009,Made For You,0.6400,0.555,7,-10.204,...,0.0981,0.3950,122.038,audio_features,1uRQzm3P0nRGnSfnuzMTpq,spotify:track:1uRQzm3P0nRGnSfnuzMTpq,https://api.spotify.com/v1/tracks/1uRQzm3P0nRG...,https://api.spotify.com/v1/audio-analysis/1uRQ...,257493,4
1,07q0QVgO56EorrSGHC48y3,I Was Made For Lovin' You,KISS,79,1979,Made For You,0.7680,0.852,4,-6.215,...,0.0723,0.8670,128.373,audio_features,07q0QVgO56EorrSGHC48y3,spotify:track:07q0QVgO56EorrSGHC48y3,https://api.spotify.com/v1/tracks/07q0QVgO56Eo...,https://api.spotify.com/v1/audio-analysis/07q0...,271240,4
2,0WIv5qV41y6YjjB9V1biuC,Made For Me,Muni Long,77,2023,Made For You,0.7960,0.468,7,-6.375,...,0.1180,0.0677,119.980,audio_features,0WIv5qV41y6YjjB9V1biuC,spotify:track:0WIv5qV41y6YjjB9V1biuC,https://api.spotify.com/v1/tracks/0WIv5qV41y6Y...,https://api.spotify.com/v1/audio-analysis/0WIv...,188000,4
3,7vF3xkCMvZjAe2nTWY0uQZ,Made For You,Jake Owen,65,2019,Made For You,0.5810,0.441,1,-6.829,...,0.1110,0.3370,82.125,audio_features,7vF3xkCMvZjAe2nTWY0uQZ,spotify:track:7vF3xkCMvZjAe2nTWY0uQZ,https://api.spotify.com/v1/tracks/7vF3xkCMvZjA...,https://api.spotify.com/v1/audio-analysis/7vF3...,238400,4
4,7zVCrzzEJU7u24sbJPXA5W,What Ifs (feat. Lauren Alaina),Kane Brown,69,2016,Made For You,0.6120,0.799,10,-4.603,...,0.1720,0.6870,125.976,audio_features,7zVCrzzEJU7u24sbJPXA5W,spotify:track:7zVCrzzEJU7u24sbJPXA5W,https://api.spotify.com/v1/tracks/7zVCrzzEJU7u...,https://api.spotify.com/v1/audio-analysis/7zVC...,188253,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2544,73IlJ59HkcXNNhLp0oBugd,Traveling,Jey Beatz,26,2024,Travel,0.7680,0.705,11,-6.026,...,0.3290,0.3780,127.918,audio_features,73IlJ59HkcXNNhLp0oBugd,spotify:track:73IlJ59HkcXNNhLp0oBugd,https://api.spotify.com/v1/tracks/73IlJ59HkcXN...,https://api.spotify.com/v1/audio-analysis/73Il...,136934,4
2545,5cWIRgigrzjTjDBNXQxhAR,Travelling Riverside Blues - 29/6/69 Top Gear;...,Led Zeppelin,38,2016,Travel,0.4760,0.780,7,-9.009,...,0.2600,0.6340,91.609,audio_features,5cWIRgigrzjTjDBNXQxhAR,spotify:track:5cWIRgigrzjTjDBNXQxhAR,https://api.spotify.com/v1/tracks/5cWIRgigrzjT...,https://api.spotify.com/v1/audio-analysis/5cWI...,310907,4
2546,4aDDlJRc1WbIlFqhc4aMgx,Calming White Noise Generator,The White Noise Travelers,62,2022,Travel,0.0616,0.350,1,-33.531,...,0.1120,0.0783,71.100,audio_features,4aDDlJRc1WbIlFqhc4aMgx,spotify:track:4aDDlJRc1WbIlFqhc4aMgx,https://api.spotify.com/v1/tracks/4aDDlJRc1WbI...,https://api.spotify.com/v1/audio-analysis/4aDD...,273500,3
2547,3ORNkXea1Uk8NdhwU4OB0B,Traveling Poor Boy,Brent Cobb,25,2016,Travel,0.6930,0.530,4,-9.928,...,0.1050,0.5190,102.795,audio_features,3ORNkXea1Uk8NdhwU4OB0B,spotify:track:3ORNkXea1Uk8NdhwU4OB0B,https://api.spotify.com/v1/tracks/3ORNkXea1Uk8...,https://api.spotify.com/v1/audio-analysis/3ORN...,214160,4


In [55]:
import numpy as np
import pandas 

columns_to_drop = ['type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms']
cleaned_spotify = unique_df.drop(columns=columns_to_drop)
cleaned_spotify.columns

Index(['Track ID', 'Track name', 'Artist name', 'Popularity score',
       'Release year', 'Genre', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'time_signature'],
      dtype='object')